# Virtex 
## End-to-End Pose Recognition

Virtex is an easy to use API for pose recognition, built on top of OpenPose, which lets you estimate and classify the poses. You can train on images having those poses or you can also train for poses in real time using the webcam. The classifier implements XG Boost Classifier to train and make predictions

### Link to Blog Post on Usage

## Getting Started and Generating Data

To keep things simple, let's just train our model to recognize standing, running and crouching. Before you start coding, make sure you are the directory of the repository to ensure we are on the same page. It is recommended to use jupyter notebook so that you can follow along easily.

Firstly, import the necessary modules.

In [7]:
import pandas as pd
from time import sleep
from api import preprocess_data, train_classifier, cross_val, generate_data, predict
from utils import combine_poses, save_obj

The generate_data() function simplifies the data generation process with just a single line of code.

In [20]:
print('Capturing standing pose in 3 seconds')
sleep(3)
generate_data('realtime', pose='standing', save_to='./example/data/', save_as='standing.csv', nb_frames=300)

Capturing standing pose in 3 seconds


As this function only generates data for a single pose at a time, we will run this function two more times for the remaining two poses i.e. crouch and running.

In [12]:
print('Capturing crouch pose in 3 seconds')
sleep(3)
generate_data('realtime', pose='crouch', save_to='./example/data/', save_as='crouch.csv', nb_frames=300)

Capturing crouch pose in 3 seconds


In [13]:
print('Capturing running pose in 3 seconds')
sleep(3)
generate_data('realtime', pose='running', save_to='./example/data/', save_as='running.csv', nb_frames=300)

Capturing running pose in 3 seconds


## Preprocessing and Preparing the Data
As you have observed that each pose has its own csv file, so we need to combine the csv files into one dataset and prepare it for the classfier.
The csv files can be combines easily using the combine_poses(*args) function from utils which accepts multiple file paths of the pose csv files as arguments and returns a dataframe.

In [21]:
dataset = combine_poses('./example/data/standing.csv','./example/data/crouch.csv','./example/data/running.csv')
# Optionally you can write the dataframe object to a csv for future reference
dataset.to_csv('./example/data/pose.csv', index=False)

Now that we have the dataset as a dataframe, we can directly send it for preprocessing using the preprocess_data() function which encodes the labels and prepares the X and Y arrays. It takes a dataframe as argument and returns two arrays i.e. X_train and y_labels and one dictionary containing a mapping from encoded integers to class labels so that we can use it display the name of the pose instead of the integer.

In [22]:
x_train, y_labels, classes = preprocess_data(dataset)

## Training
Virtex has includes an XG Boost Classifier by default and the train_classifier() function makes it super easy to train the model.

In [24]:
classifier = train_classifier(x_train, y_labels, save_as='pose', save_to='./example/classifiers')

Use the cross_val() function to get the mean accuracy and the standard deviation. It takes the model and the x_train and y_labels as arguments.

In [25]:
acc_mean, acc_std = cross_val(classifier,x_train,y_labels)
print(acc_mean,acc_std)

0.9677777777777777 0.016063146994223287


In [26]:
# We need to save the classes dictionary as it will be useful during prediction
save_obj(classes, './example/classifiers/pose.cls')

## Prediction
We are now ready to test out our model and make predictions. Just use the predict() function to make the predictions. Run the following line of code with the appropriate parameters and voila you have built a simple pose recognition program in more or less 10 lines of code.

In [27]:
predict(mode='realtime',classifier='./example/classifiers/pose.xgbc', cls_dict='./example/classifiers/pose.cls')